# 직접 해보세요!
## ebola 데이터 집합 살펴보기(155쪽)

- 하나의 열이 여러 의미를 가지고 있는 경우
- 어떤 열은 여러 가지 의미를 가지고 있을 수 있다. 예를 들어 ebola 데이터 집합의 열 중 하나인 Deaths_Guinea는 '사망자 수'와 '나라 이름'을 합쳐 만든 이름이다. 

- 다음은 ebola 데이터를 불러온 다음 0,1,2,3,10,11열의 5개 데이터만 확인한 것이다.

In [7]:
ebola = pd.read_csv('../data/country_timeseries.csv')
print(ebola.columns)

Index(['Date', 'Day', 'Cases_Guinea', 'Cases_Liberia', 'Cases_SierraLeone',
       'Cases_Nigeria', 'Cases_Senegal', 'Cases_UnitedStates', 'Cases_Spain',
       'Cases_Mali', 'Deaths_Guinea', 'Deaths_Liberia', 'Deaths_SierraLeone',
       'Deaths_Nigeria', 'Deaths_Senegal', 'Deaths_UnitedStates',
       'Deaths_Spain', 'Deaths_Mali'],
      dtype='object')


In [8]:
print(ebola.iloc[:5, [0, 1, 2, 3, 10, 11]])

         Date  Day  Cases_Guinea  Cases_Liberia  Deaths_Guinea  Deaths_Liberia
0    1/5/2015  289        2776.0            NaN         1786.0             NaN
1    1/4/2015  288        2775.0            NaN         1781.0             NaN
2    1/3/2015  287        2769.0         8166.0         1767.0          3496.0
3    1/2/2015  286           NaN         8157.0            NaN          3496.0
4  12/31/2014  284        2730.0         8115.0         1739.0          3471.0


- 일단 Date와 Day를 고정하고 나머지를 행으로 피벗하겠다. 그러면 각 나라별 사망자 수를 행으로 볼 수 있어 편리하다. 

In [9]:
ebola_long = pd.melt(ebola, id_vars=['Date', 'Day'])
print(ebola_long.head())

         Date  Day      variable   value
0    1/5/2015  289  Cases_Guinea  2776.0
1    1/4/2015  288  Cases_Guinea  2775.0
2    1/3/2015  287  Cases_Guinea  2769.0
3    1/2/2015  286  Cases_Guinea     NaN
4  12/31/2014  284  Cases_Guinea  2730.0


# 직접 해보세요!
## 열 이름 분리하고 데이터프레임에 추가하기(156쪽)

### split 메서드로 열 이름 분리하기
- Cases_Guinea와 같이 2개 이상의 의미를 가지고 있는 열 이름은 밑줄(_)을 기준으로 Cases, Guinea와 같은 방법으로 분리할 수 있다.
- 이때 열 이름을 분리하려면 split 메서드를 사용하면 된다. split 메서드는 기본적으로 공백을 기준으로 문자열을 자른다.

- Split 메서드에 '_'를 전달하면 Cases_Guinea를 Cases, Guinea로 분리할 수 있다.

In [10]:
variable_split = ebola_long.variable.str.split('_')

print(variable_split[:5])

0    [Cases, Guinea]
1    [Cases, Guinea]
2    [Cases, Guinea]
3    [Cases, Guinea]
4    [Cases, Guinea]
Name: variable, dtype: object


- 이 때 variable_split에 저장된 값의 자료형은 시리즈이고 각각의 시리즈에 저장된 값(열 이름)의 자료형은 리스트이다.

In [11]:
print(type(variable_split))

<class 'pandas.core.series.Series'>


In [12]:
print(type(variable_split[0]))

<class 'list'>


- 이때 위의 과정에서 구한 리스트의 0번째 인덱스에 담긴 문자열은 발병(Cases)과 죽음(Deaths)같은 상태를 의미하고 1번째 인덱스에 담긴 문자열은 나라 이름을 의미한다.
- 이제 이 문자열을 분리하여 데이터프레임의 새로운 열로 추가하면 된다. 다음은 get메서드를 사용하여 0, 1번째 인덱스의 데이터를 한 번에 추출한 것이다.

In [13]:
status_values = variable_split.str.get(0) 
country_values = variable_split.str.get(1)

print(status_values[:5])

0    Cases
1    Cases
2    Cases
3    Cases
4    Cases
Name: variable, dtype: object


In [14]:
print(status_values[-5:])

1947    Deaths
1948    Deaths
1949    Deaths
1950    Deaths
1951    Deaths
Name: variable, dtype: object


In [15]:
print(country_values[:5])

0    Guinea
1    Guinea
2    Guinea
3    Guinea
4    Guinea
Name: variable, dtype: object


In [16]:
print(country_values[-5:])

1947    Mali
1948    Mali
1949    Mali
1950    Mali
1951    Mali
Name: variable, dtype: object


- 다음은 위의 과정에서 분리한 문자열을 status, country라는 열 이름으로 데이터프레임에 추가한 코드이다.

In [17]:
ebola_long['status'] = status_values 
ebola_long['country'] = country_values
print(ebola_long.head())

         Date  Day      variable   value status country
0    1/5/2015  289  Cases_Guinea  2776.0  Cases  Guinea
1    1/4/2015  288  Cases_Guinea  2775.0  Cases  Guinea
2    1/3/2015  287  Cases_Guinea  2769.0  Cases  Guinea
3    1/2/2015  286  Cases_Guinea     NaN  Cases  Guinea
4  12/31/2014  284  Cases_Guinea  2730.0  Cases  Guinea


# 알아두면 좋아요!
## concat 메서드를 응용하여 데이터프레임에 열 추가하기(159쪽)

- concat 메서드를 활용하면 split메서드로 분리한 데이터를 바로 데이터프레임에 추가할 수 있다.
- 다음은 바로 위에서 실습한 내용을 concat 메서드로 바꿔서 실행한 것이다.

In [18]:
variable_split = ebola_long.variable.str.split('_', expand=True) 
variable_split.columns = ['status', 'country'] 
ebola_parsed = pd.concat([ebola_long, variable_split], axis=1)

print(ebola_parsed.head())

         Date  Day      variable   value status country status country
0    1/5/2015  289  Cases_Guinea  2776.0  Cases  Guinea  Cases  Guinea
1    1/4/2015  288  Cases_Guinea  2775.0  Cases  Guinea  Cases  Guinea
2    1/3/2015  287  Cases_Guinea  2769.0  Cases  Guinea  Cases  Guinea
3    1/2/2015  286  Cases_Guinea     NaN  Cases  Guinea  Cases  Guinea
4  12/31/2014  284  Cases_Guinea  2730.0  Cases  Guinea  Cases  Guinea
